In [15]:
from TikTokApi import TikTokApi 
import yt_dlp as yt
import whisper
import pyktok as pyk


If pyktok does not operate as expected, you may find it helpful to run the 'specify_browser' function. 'specify_browser' takes as its sole argument a string representing a browser installed on your system, e.g. "chrome," "firefox," "edge," etc.


In [19]:
def download_tiktok_audio(video_url, output_filename):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_filename,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

In [3]:
def get_tiktok_metadata(video_url):
    api = TikTokApi()
    video_id = video_url.split('/')[-1].split('?')[0]  # Extraction de l'ID vidéo à partir du lien
    video_data = api.video(id=video_id).info()
    return video_data

In [4]:
video_url = "https://www.tiktok.com/@visitberlin/video/7197066062129663237"  # Remplacez par votre lien TikTok
output_filename = "audio.mp3"

In [13]:
metadata = get_tiktok_metadata(video_url)


/var/folders/5_/q9j1rt757zv7l27qw5s892lr0000gn/T/ipykernel_83381/2372195934.py:1: RuntimeWarning: coroutine 'Video.info' was never awaited
  metadata = get_tiktok_metadata(video_url)


In [14]:
print(metadata)


<coroutine object Video.info at 0x12dd30040>


In [7]:
download_tiktok_audio(video_url,output_filename)

[TikTok] Extracting URL: https://www.tiktok.com/@visitberlin/video/7197066062129663237
[TikTok] 7197066062129663237: Downloading webpage
[info] 7197066062129663237: Downloading 1 format(s): bytevc1_1080p_1785807-2
[download] Destination: audio.mp3
[download] 100% of    2.86MiB in 00:00:07 at 405.15KiB/s 
[ExtractAudio] Destination: audio.mp3.mp3
Deleting original file audio.mp3 (pass -k to keep)


In [11]:
model = whisper.load_model("base")
result = model.transcribe("audio.mp3")
print(result["text"])

/Users/user/miniconda3/envs/travelapp/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

 Wo und was ist das mit den Spiegeln? Dieses Museum direkt am Potsdamer Platz ist ein echter Geheimtipp in Berlin. In der jeder Ecke warten hier super coole Foto-Sports und nebenbei könnt ihr etwas über die Entwicklung von Film und Fernsehen erfahren. Und wenn ihr euch jetzt immer noch fragt...


In [12]:


model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("audio.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

/Users/user/miniconda3/envs/travelapp/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

Detected language: de
Wo und was ist das mit den Spiegeln? Dieses Museum direkt am Potsdamer Platz ist ein echter Geheimtipp in Berlin. In der jeder Ecke warten hier super coole Foto-Sports und nebenbei könnt ihr etwas über die Entwicklung von Film und Fernsehen erfahren. Und wenn ihr euch jetzt immer noch fragt...


In [18]:
pyk.save_tiktok(video_url,
	        False,
            'video_data.csv')


Saved metadata for video
 https://www.tiktok.com/@visitberlin/video/7197066062129663237 
to
 /Users/user/Documents/ProjetIATiktok
